Import libraries

In [ ]:
import boto3
import json
import pandas as pd

Installing file system interface for s3

In [ ]:
!pip install s3fs

Set up the bucket name and key by entering the
bucket_name=*enter your bucket name*,
and *in_key_name=prefix/path/file*

In [ ]:
bucket_name='<your_bucket_name>'
in_key_name='<prefix>/<path>/<file>'

Reading the file content from s3

In [ ]:
s3 = boto3.resource('s3')
obj = s3.Object(bucket_name,in_key_name)
text=obj.get()['Body'].read().decode('utf-8')

Calling Comprehend API detect_sentiment for sentiment analysis

In [ ]:
client = boto3.client('comprehend')
lst=[]
for line in text.splitlines():
    line_dict=json.loads(line)
    line_dict['Sentiment']=client.detect_sentiment(Text=line_dict['text'],LanguageCode='en')['Sentiment']
    lst.append(json.dumps(line_dict))
joined_lines='\n'.join(lst)

In [ ]:
slack_df = pd.read_json(joined_lines,lines=True)

In [ ]:
slack_df.head()

In [ ]:
slack_df[['text','Sentiment']]

In [ ]:
slack_df[['text','Sentiment']].groupby(['Sentiment']).agg(['count'])

In [ ]:
slack_df[['text','Sentiment']].groupby(['Sentiment']).agg(['count']).plot(kind='pie',subplots=True)